[MovieLens Dataset Source](https://grouplens.org/datasets/movielens/100k/)

You'll need the two CSVs which are located here:

[Clean MovieLens CSVs](https://drive.google.com/drive/folders/1tJCNyCXimj6ZXc5mzmiS6701D2pWZwlG)

Go all the way to the bottom of this notebook and review steps for connecting to RAI Cloud

In [ ]:
uploaded = files.upload()
ratings = pd.read_csv(io.BytesIO(uploaded['movielens.csv']))
print(ratings.describe())

Saving movielens.csv to movielens.csv
               user           item         rating     timestamp
count  100000.00000  100000.000000  100000.000000  1.000000e+05
mean      462.48475     425.530130       3.529860  8.835289e+08
std       266.61442     330.798356       1.125674  5.343856e+06
min         1.00000       1.000000       1.000000  8.747247e+08
25%       254.00000     175.000000       3.000000  8.794487e+08
50%       447.00000     322.000000       4.000000  8.828269e+08
75%       682.00000     631.000000       4.000000  8.882600e+08
max       943.00000    1682.000000       5.000000  8.932866e+08


In [ ]:
uploaded = files.upload()
movie = pd.read_csv(io.BytesIO(uploaded['movie.csv']), sep='|')
print(movie.describe())

Saving movie.csv to movie.csv
          released       Action  ...          War      Western
count  1271.000000  1271.000000  ...  1271.000000  1271.000000
mean   1988.433517     0.154996  ...     0.043273     0.015736
std      57.335153     0.362043  ...     0.203551     0.124500
min       0.000000     0.000000  ...     0.000000     0.000000
25%    1993.000000     0.000000  ...     0.000000     0.000000
50%    1995.000000     0.000000  ...     0.000000     0.000000
75%    1996.000000     0.000000  ...     0.000000     0.000000
max    1998.000000     1.000000  ...     1.000000     1.000000

[8 rows x 19 columns]


Load in ML data .  Resilience Clusters and Shock iot


In [6]:
uploaded = files.upload()
cluster_data = pd.read_csv(io.BytesIO(uploaded['is-uncluster2.csv']))
print(cluster_data.describe())

Saving is-uncluster2.csv to is-uncluster2.csv
       nr_displaced_sclr  nr_injured_sclr  nr_assets_sclr
count         199.000000       199.000000      199.000000
mean           -1.173392         0.188716        1.844620
std             5.762550         8.401143        6.647470
min           -14.446216       -14.551736      -15.793595
25%            -5.841605        -7.538220       -3.529635
50%            -0.575889         2.486989        3.375078
75%             3.671335         8.034501        6.740190
max            12.568181        14.145166       17.865119


In [ ]:
uploaded = files.upload()
pred_data = pd.read_csv(io.BytesIO(uploaded['is-unpredict1.csv']))
print(pred_data.describe())

Saving is-unpredict1.csv to is-unpredict1.csv
          frequency  signal_attack_angle  ...  displacement  scaled_impact_level
count   1503.000000          1503.000000  ...   1503.000000          1503.000000
mean    2886.380572             6.782302  ...      0.011140           124.835943
std     3152.573137             5.918128  ...      0.013150             6.898657
min      200.000000             0.000000  ...      0.000401           103.380000
25%      800.000000             2.000000  ...      0.002535           120.191000
50%     1600.000000             5.400000  ...      0.004957           125.721000
75%     4000.000000             9.900000  ...      0.015576           129.995500
max    20000.000000            22.200000  ...      0.058411           140.987000

[8 rows x 6 columns]


In [ ]:
### Install in Rel
s = io.StringIO()
movie.to_csv(s, index=False, sep="|")
sdf = s.getvalue()

df_name = "movie_csv"
df2rel_str = f'''def config2:data = """
{sdf}"""
def config2:syntax:delim = '|'
def config2:schema:movie = "int"
def config2:schema:released = "int"
def insert:{df_name} = lined_csv[load_csv[config2]]'''

api.install_source(ctx, rai_db, rai_engine, {"movielens_movie_attrib_source":df2rel_str})

{'aborted': False,
 'actions': [{'name': 'action0',
   'result': {'type': 'InstallActionResult'},
   'type': 'LabeledActionResult'}],
 'debug_level': 0,
 'output': [],
 'problems': [],
 'type': 'TransactionResult',
 'version': 60263}

In [11]:
### Install Cluster Data in Rel (be sure to run the credentialing scripts at bottom of notebook first...)
s = io.StringIO()
cluster_data.to_csv(s, index=False, sep="|")
sdf = s.getvalue()

df_name = "cluster_csv"
df2rel_str = f'''def config3:data = """
{sdf}"""
def config3:syntax:delim = '|'
def config3:schema:ID = "string" 
def config3:schema:nr_displaced_sclr = "float" 
def config3:schema:nr_injured_sclr = "float"
def config3:schema:nr_assets_sclr = "float"
def insert:{df_name} = lined_csv[load_csv[config3]]'''

api.install_source(ctx, rai_db, rai_engine, {"cluster_source":df2rel_str})

{'aborted': False,
 'actions': [{'name': 'action0',
   'result': {'type': 'InstallActionResult'},
   'type': 'LabeledActionResult'}],
 'debug_level': 0,
 'output': [],
 'problems': [],
 'type': 'TransactionResult',
 'version': 11}

In [ ]:
ontology = f'''def person(p) = ratings_csv:user(_,p)
def movie(m,n) = movie_csv:movie(i,m) and movie_csv:title(i,n) from i
def rating(p,m,r) = person(p) and movie(m,_) and ratings_csv:user(i,p) and ratings_csv:item(i,m) and ratings_csv:rating(i,r) from i
'''
api.install_source(ctx, rai_db, rai_engine, {"movielens_ontology_source":ontology})

{'aborted': False,
 'actions': [{'name': 'action0',
   'result': {'type': 'InstallActionResult'},
   'type': 'LabeledActionResult'}],
 'debug_level': 0,
 'output': [],
 'problems': [],
 'type': 'TransactionResult',
 'version': 60264}

In [ ]:
queries = f'''@inline
def simple_recommendation[p1,m2,t] = count[p2: exists(m1,r1,r2,r3: 
    rating(p1,m1,r1) and r1=5 and                                     // Movies person 1 (p1) loved
    rating(p2,m1,r2) and not p1=p2 and r2=5 and                       // Other people that loved the same movie
	  rating(p2,m2,r3) and not rating(p1,m2,_) and r3=5 and  // Movies that other people loved and person 1 hasn't yet rated/seen
    movie(m2,t))]
'''
api.install_source(ctx, rai_db, rai_engine, {"movielens_queries_source":queries})

{'aborted': False,
 'actions': [{'name': 'action0',
   'result': {'type': 'InstallActionResult'},
   'type': 'LabeledActionResult'}],
 'debug_level': 0,
 'output': [],
 'problems': [],
 'type': 'TransactionResult',
 'version': 60265}

In [ ]:
# Cosine Similarity Source for Persons
queries = f'''
def movie_rating_count[p] = count[m: rating(p,m,_)]

def person_cosine_similarity[p1,p2](s) =
    exists(n,d:
        n=count[m: rating(p1,m, _) and rating(p2,m, _)] and
        d=sqrt[movie_rating_count[p1]]*sqrt[movie_rating_count[p2]] and
        s = n/d
    )
'''
api.install_source(ctx, rai_db, rai_engine, {"person_cosine_similarity_source":queries})

{'aborted': False,
 'actions': [{'name': 'action0',
   'result': {'type': 'InstallActionResult'},
   'type': 'LabeledActionResult'}],
 'debug_level': 0,
 'output': [],
 'problems': [],
 'type': 'TransactionResult',
 'version': 60266}

In [ ]:
# Weakly Connected Components for Persons
queries = f'''
def wcc1(x, y) = person(x) and x = y
def uedge(x, y) = person_cosine_similarity(x,y,s) and s>0.6 from s
def wcc1[x] = Min[v: uedge(x, y) and wcc1(y, v) from y]
def wcc(x,c) = Min[wcc1[x]](c)
'''
api.install_source(ctx, rai_db, rai_engine, {"wcc_source":queries})

{'aborted': False,
 'actions': [{'name': 'action0',
   'result': {'type': 'InstallActionResult'},
   'type': 'LabeledActionResult'}],
 'debug_level': 0,
 'output': [],
 'problems': [],
 'type': 'TransactionResult',
 'version': 60276}

In [ ]:
###--- Interesting query #1 ---###
### Psuedo Cypher for Path definition ==> (a:person) -[r1:rating==5]-> (m1:movie) <-[r2:rating==5]- (b:person) -[r3:rating==5]-> (m2:movie)
poi=100 # person of interest (to make this single source)
relstr = f'''def output[m2,t] = count[p2: exists(p1,m1,r1,r2,r3: p1={poi} and
    rating(p1,m1,r1) and r1=5 and                                     // Movies person 1 (p1) loved
    rating(p2,m1,r2) and not p1=p2 and r2=5 and                       // Other people that loved the same movie
	  rating(p2,m2,r3) and not rating(p1,m2,_) and r3=5 and  // Movies that other people loved and person 1 hasn't yet rated/seen
    movie(m2,t))]
'''

df_mod = pd.DataFrame(
    np.array(
        api.query(ctx, rai_db, rai_engine, relstr, None, True)["output"][0]["columns"]
    ).T.tolist()
    
    ,columns=['movie_id','movie_title','person_count']
)

df_mod['person_count']=df_mod['person_count'].astype('int')
df_mod.nlargest(5, 'person_count') #this part is clumsy to do in Rel...

,movie_id,movie_title,person_count
34,50,Star Wars,81
67,100,Fargo,53
114,174,Raiders of the Lost Ark,52
119,181,Return of the Jedi,52
38,56,Pulp Fiction,48


In [ ]:
def ss_simple_rec(person_id,nrec):
  relstr = f'def output = simple_recommendation[{person_id}]'

  df_mod = pd.DataFrame(
      np.array(
          api.query(ctx, rai_db, rai_engine, relstr, None, True)["output"][0]["columns"]
      ).T.tolist()
      
      ,columns=['movie_id','movie_title','person_count']
  )

  df_mod['person_count']=df_mod['person_count'].astype('int')
  return df_mod.nlargest(nrec, 'person_count') #this part is clumsy to do in Rel...


In [ ]:
###--- Interesting query #1 ---###
ss_simple_rec(100,5)

,movie_id,movie_title,person_count
34,50,Star Wars,81
67,100,Fargo,53
114,174,Raiders of the Lost Ark,52
119,181,Return of the Jedi,52
38,56,Pulp Fiction,48


In [ ]:
ss_simple_rec(101,10)

,movie_id,movie_title,person_count
169,313,Titanic,23
14,22,Braveheart,22
88,174,Raiders of the Lost Ark,22
55,100,Fargo,21
31,56,Pulp Fiction,19
37,69,Forrest Gump,19
173,318,Schindler's List,19
188,357,One Flew Over the Cuckoo's Nest,16
7,11,Seven (Se7en),15
9,15,Mr. Holland's Opus,15


In [ ]:
###--- Interesting query #2a ---###
### Movies and ratings for one person
poi=100 # person of interest (to make this single source)
relstr = f'''def output = p,m,t,r: p={poi} and rating(p,m,r) and movie(m,t) and r=5)
'''
qout=api.query(ctx, rai_db, rai_engine, relstr, None, True)["output"][0]["columns"]

nviz=net.Network(height='600px', width='75%',heading='')

person_id = qout[0]
movie_id = qout[1]
movie_title = qout[2]
rating = qout[3]

for e in zip(person_id, movie_id, movie_title, rating):
    p = e[0]
    m = e[1]
    mt = e[2]
    r = e[3]

    nviz.add_node(p, p, title='Person', size=30, color='#12d687', group=2)
    nviz.add_node(m, m, title=mt, size=15, color='#3a4e65', group=1)
    nviz.add_edge(p, m, width=r)

nviz.show('examplez.html')
display(HTML('examplez.html'))

In [ ]:
###--- Interesting query #2a ---###

poi=100
relstr = f'''def output = p1,m1,t1,p2,m2,t2,r1,r2: p1={poi} and r1=5 and r2=5 and p2<35 and
  rating(p1,m1,r1) and rating(p2,m1,r1) and rating(p2,m2,r2) and
  movie(m1,t1) and movie(m2,t2)
'''
qout=api.query(ctx, rai_db, rai_engine, relstr, None, True)["output"][0]["columns"]

nviz=net.Network(height='600px', width='75%',heading='')

p1 = qout[0]
m1 = qout[1]
t1 = qout[2]
r1 = qout[6]

p2 = qout[3]
m2 = qout[4]
t2 = qout[5]
r2 = qout[7]

for e in zip(p1,m1,t1,p2,m2,t2,r1,r2):
    nviz.add_node(-1*e[0], e[0], title='Person', size=30, color='#12d687', group=2)
    nviz.add_node(-1*e[3], e[3], title='Person', size=30, color='#12d687', group=2)

    nviz.add_node(e[1], e[1], title=e[2], size=15, color='#3a4e65', group=1)
    nviz.add_node(e[4], e[4], title=e[5], size=15, color='#3a4e65', group=1)

    nviz.add_edge(-1*e[0], e[1], width=e[6])
    nviz.add_edge(-1*e[3], e[4], width=e[7])

nviz.show('examplez.html')
display(HTML('examplez.html'))

In [ ]:
poi1=100
poi2=13
relstr = f'''def output = p1,p2,v: v=person_cosine_similarity[p1,p2] and p1={poi1} and p2={poi2}
'''
qout=api.query(ctx, rai_db, rai_engine, relstr, None, True)["output"][0]["columns"]
print(qout)

[[100], [13], [0.26157418189029846]]


In [ ]:
###--- Interesting query #2a ---###

cs_thresh=0.6
relstr = f'''def output = p1,p2,v,w1,w2: v=person_cosine_similarity[p1,p2] and v>{cs_thresh} and p1<p2 and w1=wcc[p1] and w2=wcc[p2]
'''
qout=api.query(ctx, rai_db, rai_engine, relstr, None, True)["output"][0]["columns"]

nviz=net.Network(height='600px', width='75%',heading='')

p1 = qout[0]
p2 = qout[1]
cs = qout[2]
w1 = qout[3]
w2 = qout[4]


for e in zip(p1,p2,cs,w1,w2):
    nviz.add_node(-1*e[0], e[0], title='Person: '+str(e[0])+'<br>WCC: '+str(e[3]), size=30, group=e[3])
    nviz.add_node(-1*e[1], e[1], title='Person: '+str(e[1])+'<br>WCC: '+str(e[4]), size=30, group=e[4])
    nviz.add_edge(-1*e[0], -1*e[1], value=e[2])

nviz.show('examplez.html')
display(HTML('examplez.html'))

In [ ]:
Steps in Configuring RAI in Notebook

Steps in Configuring RAI in Notebook

In [1]:
###--- Python Environment Setup ---###
!pip install rai-sdk
from railib import api, config, show
import functools
import json
import time
from urllib.request import HTTPError
import pandas as pd
import numpy as np
import io
from google.colab import files
!pip install pyvis
from pyvis import network as net
import networkx as nx
from IPython.core.display import display, HTML

     |████████████████████████████████| 868 kB 5.5 MB/s 
  Created wheel for ed25519: filename=ed25519-1.5-cp37-cp37m-linux_x86_64.whl size=104913 sha256=9e982637636ff6301a79613cfd98d518131bbfa5d677bb7d528dbc90e8c370d1
  Stored in directory: /root/.cache/pip/wheels/0c/fa/78/46d42bb42918d0d6ee07d94596e2f7bb7a05c1475c6c54e9ba
Successfully built ed25519


In [2]:
#@title RAI Cloud Configuration
rai_client_id = '2r7ScJbqlMskZefNadLJoBuSJydZJdn4' #@param {type:"string"}
rai_client_secret = '9gtC7bGA5K3aQAY4y-7BTGCj_f_j_nz4SjPVZ6z6QPWyADx2bMidwzNH8fwhoWQQ' #@param {type:"string"}



In [3]:
###--- Configuration File Creation ---###
rai_config_file =f'''[default]
host = azure.relationalai.com
port = 443
client_id = {rai_client_id}
client_secret = {rai_client_secret}
client_credentials_url = https://login.relationalai.com/oauth/token
'''

f=open("raiconfig",mode="w+")
f.write(rai_config_file)
f.close()

##-- Create the connection context
cfg = config.read("raiconfig")
ctx = api.Context(**cfg)

In [ ]:
###--- List available databases ---###
dbs = api.list_databases(ctx)
ldb = list(filter(lambda dbs: dbs['state'] == 'CREATED', dbs))
db_list = [d['name'] for d in ldb]
print(db_list)

['dj-test1-db', 'hersker-penguins', 'hersker-p1', 'kglkg', 'BH-Test', 'demo', 'un-demo-20211018', 'mpo_z1', 'dj-test2-db', 'imdb-gregory', 'IS-my-first-database2', 'vvDb3', 'gregory-imdb-2', 'gregory-imdb-3', 'gregory-imdb-5', 'gregory-dummy-db', 'vvDb5', 'is-UN3-db']


In [4]:
###--- List available compute engines ---###
engines = api.list_engines(ctx)
le = list(filter(lambda engines: engines['state'] == 'PROVISIONED', engines))
engine_list = [d['name'] for d in le]
print(engine_list)

['dj_engine', 'is-221121']


In [ ]:
###! Delete an engine
#api.delete_engine(ctx, rai_engine)
#api.delete_engine(ctx, 'is_un1_1636551291')

{'status': {'message': 'compute "is_un1_1636551291" deleted successfully',
  'name': 'is_un1_1636551291',
  'state': 'DELETING'}}

In [10]:
#@title Database & Compute Engine { run: "auto" }
rai_db = "is-UN3-db" #@param {type:"string"}
rai_engine = "is-221121" #@param {type:"string"}
rai_engine_size = "S" #@param ["XS", "S", "M", "L", "XL"]

## Check Engine and DB Names:
print("rai_db: "+rai_db)
print("rai_engine: "+rai_engine)

rai_db: is-UN3-db
rai_engine: is-221121


In [ ]:
### Create a compute engine (if necessary)
#! Passing engine size does not work currently
epoch_time = int(time.time())
rai_engine += "_"+str(epoch_time)

print(rai_engine)

try:
    response = api.create_engine(ctx, rai_engine)
    print(json.dumps(response, indent=2))
except HTTPError as e:
    show.http_error(e)

mpo_z1_1636473613
{
  "compute": {
    "account_name": "relationalai-team-sales",
    "id": "6131f99c-d0d0-184a-d0f2-61d670b0b8d0",
    "name": "mpo_z1_1636473613",
    "size": "XS",
    "region": "us-east",
    "state": "REQUESTED",
    "created_by": "BKg1J1q6V0w9oK0jYktxnwOrMSNWz9mH@clients",
    "requested_on": "2021-11-09T16:00:13.377Z"
  }
}


In [ ]:
### Create and overwrite a database (if necessary)
try:
    response = api.create_database(ctx, rai_db, rai_engine, overwrite=True)
    print(json.dumps(response, indent=2))
except HTTPError as e:
    show.http_error(e)

{
  "output": [],
  "version": 10002,
  "problems": [],
  "actions": [],
  "debug_level": 0,
  "aborted": false,
  "type": "TransactionResult"
}


###### **ML script 1.**  Model to  to create clusters for resilience status based on key metircs of numbers displaced, injured and volume of assets deployed as a result of an incident. This will help policy makers allocate appropriate levels of support based on scientific groupings rather than pure gut feel.


######***STEP 1:*** Firstly we need to  assign the input features for the cluster build to a Relation object in Rel. We named this relation'features'.  Note that triple ' are used to denote that we are using a multiple line string.The api then writes the results to a source in the database called 'features_source'. Because we used the api.install_source parameter.
###### This is persisted in the Knowledge Graph as an EDB Relation (see Rel documentation on working with EDB relations https://docs.relational.ai/rel/concepts/updates/)



In [12]:
queries = '''
def features = cluster_csv[col] for col in {:nr_displaced_sclr; :nr_injured_sclr; :nr_assets_sclr}
'''
api.install_source(ctx, rai_db, rai_engine, {"features_source":queries})

{'aborted': False,
 'actions': [{'name': 'action0',
   'result': {'type': 'InstallActionResult'},
   'type': 'LabeledActionResult'}],
 'debug_level': 0,
 'output': [],
 'problems': [],
 'type': 'TransactionResult',
 'version': 12}

######***STEP 2:*** Next, we can query the inout feature relation using python functions that match what we are used to in the EDA cycle.  In this case, we apply a Describe function to list the datasets summary statistics. Note the api call ....

In [13]:
api.query(ctx, rai_db, rai_engine, "describe[features]")

{'aborted': False,
 'actions': [{'name': 'action0',
   'result': {'output': [], 'type': 'QueryActionResult'},
   'type': 'LabeledActionResult'}],
 'debug_level': 0,
 'output': [{'columns': [[17.865119266491803]],
   'rel_key': {'keys': [':nr_assets_sclr', ':max'],
    'name': 'output',
    'type': 'RelKey',
    'values': ['Float64']},
   'type': 'Relation'},
  {'columns': [[2.4869893278351403]],
   'rel_key': {'keys': [':nr_injured_sclr', ':percentile50'],
    'name': 'output',
    'type': 'RelKey',
    'values': ['Float64']},
   'type': 'Relation'},
  {'columns': [[199]],
   'rel_key': {'keys': [':nr_injured_sclr', ':count'],
    'name': 'output',
    'type': 'RelKey',
    'values': ['Int64']},
   'type': 'Relation'},
  {'columns': [[-14.4462162454763]],
   'rel_key': {'keys': [':nr_displaced_sclr', ':min'],
    'name': 'output',
    'type': 'RelKey',
    'values': ['Float64']},
   'type': 'Relation'},
  {'columns': [[5.762550358343904]],
   'rel_key': {'keys': [':nr_displaced_sclr', 

######***STEP 3:*** We now run the Cluster models, based on Rel's MLPacks implementation.  In this example, four parameters are passed: 1) the number of k's 2) the input dataset 3) the location of the unique ID (note: this must be in the first column location) and  4) the model parameters .  See details at  https://docs.relational.ai/rel/ref/lib/ml/

In [14]:
queries = '''
def clusters = mlpack_kmeans[4,features,1,{("algorithm","pelleg-moore");("seed","123")}]
def centroids = mlpack_kmeans_centroids[4,features,1,{("algorithm","pelleg-moore");("seed","123")}]
'''
api.install_source(ctx, rai_db, rai_engine, {"kmeans_source":queries})

#print(queries)


{'aborted': False,
 'actions': [{'name': 'action0',
   'result': {'type': 'InstallActionResult'},
   'type': 'LabeledActionResult'}],
 'debug_level': 0,
 'output': [],
 'problems': [],
 'type': 'TransactionResult',
 'version': 13}

***STEP 4:*** Use the same api call applied in Step 2, this time to query the clusters generated.

In [ ]:
api.query(ctx, rai_db, rai_engine, "table[clusters]")

{'aborted': False,
 'actions': [{'name': 'action0',
   'result': {'output': [], 'type': 'QueryActionResult'},
   'type': 'LabeledActionResult'}],
 'debug_level': 0,
 'output': [{'columns': [['application/vnd.rel.relation.table']],
   'rel_key': {'keys': [':MIME'],
    'name': 'output',
    'type': 'RelKey',
    'values': ['String']},
   'type': 'Relation'},
  {'columns': [[1,
     2,
     3,
     4,
     5,
     6,
     7,
     8,
     9,
     10,
     11,
     12,
     13,
     14,
     15,
     16,
     17,
     18,
     19,
     20,
     21,
     22,
     23,
     24,
     25,
     26,
     27,
     28,
     29,
     30,
     31,
     32,
     33,
     34,
     35,
     36,
     37,
     38,
     39,
     40,
     41,
     42,
     43,
     44,
     45,
     46,
     47,
     48,
     49,
     50,
     51,
     52,
     53,
     54,
     55,
     56,
     57,
     58,
     59,
     60,
     61,
     62,
     63,
     64,
     65,
     66,
     67,
     68,
     69,
     70,
     71,

***STEP 5***: U

In [15]:
relstr = f'def output = count[a: clusters(a,b)] for b'

df_mod = pd.DataFrame(
    np.array(
        api.query(ctx, rai_db, rai_engine, relstr, None, True)["output"][0]["columns"]
    ).T.tolist()
    
    ,columns=['cluster_id','cluster_count']
)

df_mod['cluster_id']=df_mod['cluster_id'].astype('int')
df_mod['cluster_count']=df_mod['cluster_count'].astype('int')
df_mod

,cluster_id,cluster_count
0,1,50
1,2,53
2,3,47
3,4,49


In [ ]:
relstr = f'def output = centroids'

df_mod = pd.DataFrame(
    np.array(
        api.query(ctx, rai_db, rai_engine, relstr, None, True)["output"][0]["columns"]
    ).T.tolist()
    
    ,columns=['cluster_id','x','y','z']
)

df_mod['cluster_id']=df_mod['cluster_id'].astype('int')
df_mod['x']=df_mod['x'].astype('float')
df_mod['y']=df_mod['y'].astype('float')
df_mod['z']=df_mod['z'].astype('float')
df_mod

,cluster_id,x,y,z
0,1,-7.707840,2.754621,-6.318110
1,2,4.437593,-2.690495,8.540878
2,3,1.825605,-9.168194,7.596393
3,4,8.805622,4.127864,-9.310958


Visualise Resilience Clusters

In [ ]:
from __future__ import division
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(111, projection='3d')
# Point generation by using normal standard distribution
# 3D plotting and its projection to xy plane
z_offset = 3

# Cluster of points #1
# --------------------
coordinates_c1 = np.array([2, 1, 5])  # center point of the cluster
cluster1 = 0.15 * np.random.standard_normal((50,3)) + coordinates_c1
# Plotting 3D points
ax.plot(cluster1[:, 0], cluster1[:, 1], cluster1[:, 2],
'ko', alpha=0.6, label='Population Displaced')
# Plotting projection (notice that the Z coordinates are set to zero)
ax.plot(cluster1[:, 0], cluster1[:, 1], np.zeros_like(cluster1[:, 2])+z_offset,
'ko')

# Cluster of points #2
# --------------------
coordinates_c2 = np.array([3.5, 2.5, 6])  # center point of the cluster
cluster2 = 0.3 * np.random.standard_normal((50,3)) + coordinates_c2
# Plotting 3D points
ax.plot(cluster2[:, 0], cluster2[:, 1], cluster2[:, 2],
'ro', alpha=0.6, label='Number of Injuries')
# Plotting projection (notice that the Z coordinates are set to zero)
ax.plot(cluster2[:, 0], cluster2[:, 1], np.zeros_like(cluster2[:, 2])+z_offset,
'ro')

# Cluster of points #3
# --------------------
coordinates_c3 = np.array([6, 3, 7])  # center point of the cluster
cluster3 = 0.4 * np.random.standard_normal((50, 3)) + coordinates_c3
ax.plot(cluster3[:, 0], cluster3[:, 1], cluster3[:, 2],
'go', alpha=0.6, label='Assets Deployed')
# Plotting projection (notice that the Z coordinates are set to zero)
ax.plot(cluster3[:, 0], cluster3[:, 1], np.zeros_like(cluster3[:, 2])+z_offset,
'go')

# Sphere surface #1
# ------------------
u1 = np.linspace(0, 2 * np.pi, 100)
v1 = np.linspace(0, np.pi, 100)
x_sphere_1 = 1   * np.outer(np.cos(u1), np.sin(v1)) + coordinates_c1[0]
y_sphere_1 = 0.5 * np.outer(np.sin(u1), np.sin(v1)) + coordinates_c1[1]
z_sphere_1 = 1.5 * np.outer(np.ones(np.size(u1)), np.cos(v1)) + coordinates_c1[2]
ax.plot_surface(x_sphere_1, y_sphere_1, z_sphere_1,
rstride=10, cstride=10, linewidth=0.1, color='b', alpha=0.1)
# Sphere surface #2
# ------------------
u2 = np.linspace(0, 2 * np.pi, 100)
v2 = np.linspace(0, np.pi, 100)
x_sphere_2 = 1.5 * np.outer(np.cos(u2), np.sin(v2)) + coordinates_c2[0]
y_sphere_2 = 1   * np.outer(np.sin(u2), np.sin(v2)) + coordinates_c2[1]
z_sphere_2 = 1.8 * np.outer(np.ones(np.size(u2)), np.cos(v2)) + coordinates_c2[2]
ax.plot_surface(x_sphere_2, y_sphere_2, z_sphere_2,
rstride=10, cstride=10, linewidth=0.1, color='r', alpha=0.1)

# Sphere surface #3
# -----------------
u3 = np.linspace(0, 2 * np.pi, 100)
v3 = np.linspace(0, np.pi, 100)
x_sphere_3 = 1.5 * np.outer(np.cos(u3), np.sin(v3)) + coordinates_c3[0]
y_sphere_3 = 1   * np.outer(np.sin(u3), np.sin(v3)) + coordinates_c3[1]
z_sphere_3 = 2   * np.outer(np.ones(np.size(u3)), np.cos(v3)) + coordinates_c3[2]
ax.plot_surface(x_sphere_3, y_sphere_3, z_sphere_3,
rstride=10, cstride=10, linewidth=0.1, color='g', alpha=0.1)

# Limits of the 3D representation
ax.set_xlim3d(0, 8)
ax.set_ylim3d(0, 4)
ax.set_zlim3d(z_offset, 9)
# Labels for the legend
ax.set_xlabel(u'Population Displaced')
ax.set_ylabel(u'Number of Injuries')
ax.set_zlabel(u'Assets Deployed')
# Show legend
ax.legend()
# If we want to save the figure into PDF o PNG
# fig.savefig(r'/cluster-projection.pdf')
# fig.savefig(r'/cluster-projection.png',dpi=150)
plt.show()

In [ ]:
!pip install plotly

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objs as go
from plotly import tools
from plotly.subplots import make_subplots
import plotly.offline as py

In [ ]:
 #3d scatterplot using matplotlib

fig = plt.figure(figsize = (15,15))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x[y_clusters == 0,0],x[y_clusters == 0,1],x[y_clusters == 0,2], s = 40 , color = 'blue', label = "cluster 1")
ax.scatter(x[y_clusters == 1,0],x[y_clusters == 1,1],x[y_clusters == 1,2], s = 40 , color = 'orange', label = "cluster 2")
ax.scatter(x[y_clusters == 2,0],x[y_clusters == 2,1],x[y_clusters == 2,2], s = 40 , color = 'green', label = "cluster 3")
ax.scatter(x[y_clusters == 3,0],x[y_clusters == 3,1],x[y_clusters == 3,2], s = 40 , color = '#D12B60', label = "cluster 4")
ax.set_xlabel('Population Displaced-->')
ax.set_ylabel('Number of Injuries-->')
ax.set_zlabel('Assets Deployed-->')
ax.legend()
plt.show()

In [ ]:
# 3d scatterplot using plotly
#Scene = dict(xaxis = dict(title  = 'Population Displaced -->'),yaxis = dict(title  = 'Number of Injuries--->'),zaxis = dict(title  = 'Assets Deployed-->'))


#labels = model.labels_
#trace = go.Scatter3d(x=x[:, 0], y=x[:, 1], z=x[:, 2], mode='markers',marker=dict(color = labels, size= 10, line=dict(color= 'black',width = 10)))
#layout = go.Layout(margin=dict(l=0,r=0),scene = Scene,height = 800,width = 800)
#data = [trace]
#fig = go.Figure(data = data, layout = layout)
#fig.show()